In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

np.random.seed(0)

In [3]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

In [4]:
from tensorflow_federated.proto.v0 import computation_pb2 as pb


NUM_CLIENTS = 10
NUM_EPOCHS = 5
SHUFFLE_BUFFER = 100

def preprocess(dataset):

  def map_fn(element):
    return [tf.reshape(element['pixels'], [-1, 784]),
        tf.reshape(element['label'], [-1, 1])]

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).map(map_fn)

In [5]:
example_dataset = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[0])
preprocessed_example_dataset = preprocess(example_dataset)

In [6]:
def create_keras_model():
  return tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer='zeros'),
      tf.keras.layers.Softmax(),
  ])
  
def model_fn():
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=preprocessed_example_dataset.element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [7]:
model = create_keras_model()

input_spec = (tf.TensorSpec(shape=(1,784), dtype=tf.float32, name=None), 
              tf.TensorSpec(shape=(1,1), dtype=tf.int32, name=None))
print(input_spec)
print(preprocessed_example_dataset.element_spec)

functional_model = tff.learning.models.functional_model_from_keras(keras_model=model, loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(),input_spec=input_spec)

def tff_model_fn() -> tff.learning.Model:
    return tff.learning.models.model_from_functional(functional_model)

(TensorSpec(shape=(1, 784), dtype=tf.float32, name=None), TensorSpec(shape=(1, 1), dtype=tf.int32, name=None))
(TensorSpec(shape=(1, 784), dtype=tf.float32, name=None), TensorSpec(shape=(1, 1), dtype=tf.int32, name=None))


In [8]:
tff.learning.models.save_functional_model(functional_model=functional_model, path='tmp_dir')
saved_functional_model = tff.learning.models.load_functional_model(
                "tmp_dir"
            )
def saved_tff_model_fn() -> tff.learning.Model:
    
    return tff.learning.models.model_from_functional(saved_functional_model)

INFO:tensorflow:Assets written to: tmp_dir/assets


INFO:tensorflow:Assets written to: tmp_dir/assets


In [9]:
import os
OUTPUT_DIR = 'some_dir'
train_output_managers = [tff.program.LoggingReleaseManager()]
evaluation_output_managers = [tff.program.LoggingReleaseManager()]
model_output_manager = tff.program.LoggingReleaseManager()

# # there is an issue with this, it causes and error for some reason
# summary_dir = os.path.join(OUTPUT_DIR, "summary")
# tensorboard_manager = tff.program.TensorBoardReleaseManager(summary_dir)
# train_output_managers.append(tensorboard_manager)

# # there is an issue with this, it causes and error for some reason
# csv_path = os.path.join(OUTPUT_DIR, "evaluation_metrics.csv")
# csv_manager = tff.program.CSVFileReleaseManager(csv_path)
# evaluation_output_managers.append(csv_manager)

# # there is an issue with this, it causes and error for some reason
# program_state_dir = os.path.join(OUTPUT_DIR, "program_state")
# program_state_manager = tff.program.FileProgramStateManager(program_state_dir)


In [10]:
#emnist_train, emnist_test
preprocessed_example_dataset

<MapDataset element_spec=(TensorSpec(shape=(1, 784), dtype=tf.float32, name=None), TensorSpec(shape=(1, 1), dtype=tf.int32, name=None))>

In [11]:
number_of_clients = 3 
total_rounds = 10
train_datasets = [preprocess(emnist_train.create_tf_dataset_for_client(i)) for i in emnist_train.client_ids]
test_datasets = [preprocess(emnist_test.create_tf_dataset_for_client(i)) for i in emnist_test.client_ids]
train_data_source = tff.program.DatasetDataSource(train_datasets)
evaluation_data_source = tff.program.DatasetDataSource(test_datasets)

In [12]:
import functools
from tensorflow_federated.python.program import value_reference


async def tff_train_federated(
    initialize: tff.Computation,
    train: tff.Computation,
    train_data_source: tff.program.FederatedDataSource,
    evaluation: tff.Computation,
    evaluation_data_source: tff.program.FederatedDataSource,
    total_rounds: int,
    number_of_clients: int,
    train_output_managers,
    evaluation_output_managers,
    model_output_manager: tff.program.ReleaseManager,
    program_state_manager: tff.program.ProgramStateManager,
) -> None:
    tff.program.check_in_federated_context()

    # The program state manager is not working but maybe we could drop it
    if program_state_manager is not None:
        structure = initialize()
        program_state, version = await program_state_manager.load_latest(structure)
    else:
        program_state = None

    if program_state is not None:

        state, start_round = program_state
    else:
        state = initialize()
        start_round = 1

    # state = initialize()
    # start_round = 1
    async with tff.async_utils.ordered_tasks() as tasks:

        train_data_iterator = train_data_source.iterator()

        for round_number in range(start_round, total_rounds + 1):
            tasks.add_callable(
                functools.partial(
                    print, f"Running round {round_number} of training" 
                )
            )

            train_data = train_data_iterator.select(number_of_clients)
            output = train(state, train_data)
            state = output.state
            metrics = output.metrics

            if train_output_managers is not None:
                _, metrics_type = train.type_signature.result
                # tasks.add_all(
                #     *[m.release(metrics, metrics_type, round_number) for m in train_output_managers]
                # )
                materialized_value = await value_reference.materialize_value(metrics)

                tasks.add_callable(
                    functools.partial(
                        print, str(materialized_value) 
                    )
                )

            # This is not working
            # if program_state_manager is not None:
            #     program_state = (state, start_round)
            #     tasks.add(program_state_manager.save(program_state, round_number))

        evaluation_data_iterator = evaluation_data_source.iterator()
        evaluation_data = evaluation_data_iterator.select(number_of_clients)
        evaluation_metrics = evaluation(state, evaluation_data)

        # if evaluation_output_managers is not None:
        #     evaluation_metrics_type = evaluation.type_signature.result
        #     tasks.add_all(*[
        #         m.release(evaluation_metrics, evaluation_metrics_type, round_number)
        #         for m in train_output_managers
        #     ])

        # if model_output_manager is not None:
        #     state_type, _ = train.type_signature.result
        #     tasks.add(model_output_manager.release(state, state_type))



In [13]:
# KERAS MODEL 
import asyncio
context = tff.backends.native.create_local_async_python_execution_context()
context = tff.program.NativeFederatedContext(context)
tff.framework.set_default_context(context)

iterative_process = tff.learning.algorithms.build_unweighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))
initialize = iterative_process.initialize
train = iterative_process.next
evaluation = tff.learning.build_federated_evaluation(model_fn)
import logging
logger = logging.getLogger()

logger.setLevel(logging.INFO)
logging.info("Test")

asyncio.run(
        tff_train_federated(
            initialize=initialize,
            train=train,
            train_data_source=train_data_source,
            evaluation=evaluation,
            evaluation_data_source=evaluation_data_source,
            total_rounds=total_rounds,
            number_of_clients=number_of_clients,
            train_output_managers=train_output_managers,
            evaluation_output_managers=evaluation_output_managers,
            model_output_manager=model_output_manager,
            program_state_manager=None,
            # program_state_manager=program_state_manager,
        )
    )

INFO:root:Test
/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/site-packages/tensorflow_federated/python/program/native_platform.py:97: RuntimeWarning: coroutine '_create_structure_of_coro_references.<locals>._to_structure' was never awaited
  element = _create_structure_of_coro_references(element_coro, element_type)
/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/site-packages/tensorflow_federated/python/program/native_platform.py:97: RuntimeWarning: coroutine '_create_structure_of_coro_references.<locals>._get_item' was never awaited
  element = _create_structure_of_coro_references(element_coro, element_type)
/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/site-packages/tensorflow_federated/python/program/native_platform.py:96: RuntimeWarning: coroutine '_create_structure_of_coro_references.<locals>._get_item' was never awaited
  element_coro = _get_item(shared_awaitable, index)
/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/site-packages/tensorflow_federated/python/program/native_platf

Running round 1 of training
OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('sparse_categorical_accuracy', 0.112933755), ('loss', 10.612101), ('num_examples', 1585), ('num_batches', 1585)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_count', ())])), ('finalizer', ())])
Running round 2 of training
OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('sparse_categorical_accuracy', 0.115611814), ('loss', 10.417141), ('num_examples', 1185), ('num_batches', 1185)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_count', ())])), ('finalizer', ())])
Running round 3 of training
OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('sparse_categorical_accuracy', 0.17106918), ('loss', 7.437316), ('num_examples', 1590), ('num_batches', 1590)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_count', ())])), ('finalizer', ())])
Running round 4 of training

/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/asyncio/tasks.py:256: RuntimeWarning: coroutine '_create_structure_of_coro_references.<locals>._get_item' was never awaited
  result = coro.send(None)
/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/asyncio/tasks.py:256: RuntimeWarning: coroutine '_create_structure_of_coro_references.<locals>._to_structure' was never awaited
  result = coro.send(None)
/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/asyncio/tasks.py:256: RuntimeWarning: coroutine 'NativeFederatedContext.invoke.<locals>._invoke' was never awaited
  result = coro.send(None)


In [14]:
# FUNCTIONAL KERAS MODEL 
import asyncio
context = tff.backends.native.create_local_async_python_execution_context()
context = tff.program.NativeFederatedContext(context)
tff.framework.set_default_context(context)

iterative_process = tff.learning.algorithms.build_unweighted_fed_avg(
    tff_model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))
initialize = iterative_process.initialize
train = iterative_process.next
evaluation = tff.learning.build_federated_evaluation(tff_model_fn)

asyncio.run(
        tff_train_federated(
            initialize=initialize,
            train=train,
            train_data_source=train_data_source,
            evaluation=evaluation,
            evaluation_data_source=evaluation_data_source,
            total_rounds=total_rounds,
            number_of_clients=number_of_clients,
            train_output_managers=train_output_managers,
            evaluation_output_managers=evaluation_output_managers,
            model_output_manager=model_output_manager,
            program_state_manager=None,
            # program_state_manager=program_state_manager,
        )
    )

Running round 1 of training
OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('loss', 12.598344)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_count', ())])), ('finalizer', ())])
Running round 2 of training
OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('loss', 12.957993)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_count', ())])), ('finalizer', ())])
Running round 3 of training
OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('loss', 11.286803)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_count', ())])), ('finalizer', ())])
Running round 4 of training
OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('loss', 7.2167797)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_count', ())])), ('finalizer', ())])
Running round 5 of training
OrderedDict([('distributor', ()), ('client_w

In [20]:
# FUNCTIONAL KERAS MODEL FROM A SAVED MODEL
import asyncio
context = tff.backends.native.create_local_async_python_execution_context()
context = tff.program.NativeFederatedContext(context)
tff.framework.set_default_context(context)
import logging
logger = logging.getLogger()

logger.setLevel(logging.WARN)
iterative_process = tff.learning.algorithms.build_unweighted_fed_avg(
    saved_tff_model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))
initialize = iterative_process.initialize
train = iterative_process.next
evaluation = tff.learning.build_federated_evaluation(saved_tff_model_fn)

asyncio.run(
        tff_train_federated(
            initialize=initialize,
            train=train,
            train_data_source=train_data_source,
            evaluation=evaluation,
            evaluation_data_source=evaluation_data_source,
            total_rounds=total_rounds,
            number_of_clients=number_of_clients,
            train_output_managers=train_output_managers,
            evaluation_output_managers=evaluation_output_managers,
            model_output_manager=model_output_manager,
            program_state_manager=None,
            # program_state_manager=program_state_manager,
        )
    )

AttributeError: 'Tensor' object has no attribute 'numpy'